In [2]:
import requests
import json
import pandas as pd
import datetime as dt
# noinspection SpellCheckingInspection

In [3]:

class ReadDB(object):

    def __init__(self):
        """This object was created to extract data from API XM"""

        self.url = "http://servapibi.xm.com.co/hourly"
        self.connection = None
        self.request = ''
        self.lectura = requests.get(r'https://raw.githubusercontent.com/EquipoAnaliticaXM/API_XM/master/pydataxm/metricasAPI.json').json()
        self.inventario_metricas = json.loads(self.lectura)
    
    def get_collections(self, coleccion):

        return self.inventario_metricas[coleccion]

    def request_data(self, coleccion, metrica, start_date, end_date, filtros):
        """ request public server data from XM by the API
        Args:
            coleccion: one of the set of variables availables at self.get_collections()
            metrica:one of this variables available in "ListadoMetricas", you have to enter MetricID
            start_date: start date consult data using YYYY-MM-DD format
            end_date: end date consult data using YYYY-MM-DD format
            filter: optional parameter, list of values to filter data
        Returns: DataFrame with the raw Data
        """
        if type(filtros)==list:
            self.filtros = filtros
        else:
            print('Los filtros deben ingresarse como una lista de valores')
            self.filtros = []
        if coleccion not in self.inventario_metricas.keys():
            print('No existe la colección {}'.format(coleccion))
            return pd.DataFrame()
        if metrica > len(self.inventario_metricas[coleccion]):
            print('No existe la metrica')
            return pd.DataFrame()

        if self.inventario_metricas[coleccion][metrica][3] == 'Horaria':
            end = end_date
            condition = True
            aux = True
            data = None
            while condition:
                if (start_date - end_date).days < 30:
                    end = start_date + dt.timedelta(29)
                if end > end_date:
                    end = end_date
                self.request = {"MetricId": coleccion,
                                "StartDate": "{}".format(str(start_date)),
                                "EndDate": "{}".format(str(end)),
                                'Entity': self.inventario_metricas[coleccion][metrica][2],
                                "Filter": self.filtros}

                self.connection = requests.post(self.url, json=self.request)

                data_json = json.loads(self.connection.content)

                temporal_data = pd.json_normalize(data_json['Items'], 'HourlyEntities', 'Date', sep='_')

                if data is None:
                    data = temporal_data.copy()
                else:
                    data = data.append(temporal_data, ignore_index=True)
                start_date = start_date + dt.timedelta(30)

                if end == end_date:
                    aux = False
                condition = ((end - start_date).days > 30 | (end - end_date).days != 0) | aux
        elif self.inventario_metricas[coleccion][metrica][3] == 'Diaria' and coleccion == 'CapEfecNeta':
            end = end_date
            condition = True
            aux = True
            data = None
            while condition:
                if (start_date - end_date).days < 1:
                    end = start_date + dt.timedelta(0)
                if end > end_date:
                    end = end_date
                self.request = {"MetricId": coleccion,
                                "StartDate": "{}".format(str(start_date)),
                                "EndDate": "{}".format(str(end)),
                                'Entity': self.inventario_metricas[coleccion][metrica][2],
                                "Filter": self.filtros}
                self.url = self.url.replace('hourly', 'daily')
                self.connection = requests.post(self.url, json=self.request)

                data_json = json.loads(self.connection.content)

                temporal_data = pd.json_normalize(data_json['Items'], 'DailyEntities', 'Date', sep='_')

                if data is None:
                    data = temporal_data.copy()
                else:
                    data = data.append(temporal_data, ignore_index=True)
                start_date = start_date + dt.timedelta(1)

                if end == end_date:
                    aux = False
                condition = ((end - start_date).days > 1 | (end - end_date).days != 0) | aux
        elif self.inventario_metricas[coleccion][metrica][3] == 'Diaria':
            end = end_date
            condition = True
            aux = True
            data = None
            while condition:
                if (start_date - end_date).days < 30:
                    end = start_date + dt.timedelta(29)
                if end > end_date:
                    end = end_date

                self.request = {"MetricId": coleccion,
                                "StartDate": "{}".format(str(start_date)),
                                "EndDate": "{}".format(str(end)),
                                'Entity': self.inventario_metricas[coleccion][metrica][2],
                                "Filter": self.filtros}
                self.url = self.url.replace('hourly', 'daily')
                self.connection = requests.post(self.url, json=self.request)
                data_json = json.loads(self.connection.content)
                temporal_data = pd.json_normalize(data_json['Items'], 'DailyEntities', 'Date', sep='_')
                if data is None:
                    data = temporal_data.copy()
                else:
                    data = data.append(temporal_data, ignore_index=True)

                start_date = start_date + dt.timedelta(30)
                if end == end_date:
                    aux = False
                condition = ((end - start_date).days > 29 | (end - end_date).days != 0) | aux

        elif self.inventario_metricas[coleccion][metrica][3] == 'Mensual':
            
            end = end_date
            condition = True
            aux = True
            data = None
            while condition:
                if (start_date - end_date).days < 732:
                    end = start_date + dt.timedelta(731)
                if end > end_date:
                    end = end_date

                self.request = {"MetricId": coleccion,
                                "StartDate": "{}".format(str(start_date)),
                                "EndDate": "{}".format(str(end)),
                                'Entity': self.inventario_metricas[coleccion][metrica][2],
                                "Filter": self.filtros}
                self.url = self.url.replace('hourly', 'monthly')
                self.connection = requests.post(self.url, json=self.request)
                data_json = json.loads(self.connection.content)
                temporal_data = pd.json_normalize(data_json['Items'], 'MonthlyEntities', sep='_')
                if data is None:
                    data = temporal_data.copy()
                else:
                    data = data.append(temporal_data, ignore_index=True)

                start_date = start_date + dt.timedelta(732)
                if end == end_date:
                    aux = False
                condition = ((end - start_date).days > 731 | (end - end_date).days != 0) | aux

        elif self.inventario_metricas[coleccion][metrica][3] == 'Anual':
            
            end = end_date
            condition = True
            aux = True
            data = None
            while condition:
                if (start_date - end_date).days < 366:
                    end = start_date + dt.timedelta(365)
                if end > end_date:
                    end = end_date

                self.request = {"MetricId": coleccion,
                                "StartDate": "{}".format(str(start_date)),
                                "EndDate": "{}".format(str(end)),
                                'Entity': self.inventario_metricas[coleccion][metrica][2],
                                "Filter": self.filtros}
                self.url = self.url.replace('hourly', 'annual')
                self.connection = requests.post(self.url, json=self.request)
                data_json = json.loads(self.connection.content)
                temporal_data = pd.json_normalize(data_json['Items'], 'AnnualEntities', 'Code', sep='_')
                if data is None:
                    data = temporal_data.copy()
                else:
                    data = data.append(temporal_data, ignore_index=True)

                start_date = start_date + dt.timedelta(366)
                if end == end_date:
                    aux = False
                condition = ((end - start_date).days > 365 | (end - end_date).days != 0) | aux


        elif self.inventario_metricas[coleccion][metrica][3] == 'Lista' and not coleccion=='ListadoMetricas':
            self.request = {"MetricId": coleccion,
                                # "StartDate": "{}".format(str(start_date)),
                                # "EndDate": "{}".format(str(end)),
                                'Entity': self.inventario_metricas[coleccion][metrica][2]}
            self.url = self.url.replace('hourly', 'lists')
            self.connection = requests.post(self.url, json=self.request)
            data_json = json.loads(self.connection.content)
            data = pd.json_normalize(data_json['Items'], 'ListEntities','Date', sep='_')
        
        elif self.inventario_metricas[coleccion][metrica][3] == 'Lista' and coleccion=='ListadoMetricas':
            self.request = {"MetricId": coleccion,
                    # "StartDate": "{}".format(str(start_date)),
                    # "EndDate": "{}".format(str(end)),
                    'Entity': self.inventario_metricas[coleccion][metrica][2]}
            self.url = self.url.replace('hourly', 'lists')
            self.connection = requests.post(self.url, json=self.request)
            data_json = json.loads(self.connection.content)
            data = pd.json_normalize(data_json['Items'], 'Values', sep='_')
        
        
        cols = data.columns
        for col in cols:
            data[col] = pd.to_numeric(data[col],errors='ignore')
        if 'Date' or 'date' in cols:
            data['Date'] = pd.to_datetime(data['Date'],errors='ignore')
        data.drop(columns=['Id'],inplace=True)

        return data

In [4]:
consult = ReadDB()


In [5]:
df1 = consult.request_data("Gene", 1, dt.date(2020, 1, 1), dt.date(2020, 1, 5),filtros=['TYP1','TYP2'])
df1

,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,Values_Hour09,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,TYP1,19707.87,19710.22,19656.49,19699.23,19725.60,19709.97,19727.99,19487.47,19014.10,...,18812.29,18986.84,19222.01,19277.52,19218.91,19618.68,19717.34,19674.21,19670.40,2020-01-01
1,TYP2,25243.54,22580.11,22542.15,22545.30,22514.44,22580.73,22635.55,22711.01,30640.19,...,30414.93,30650.39,31004.22,31112.21,31154.00,31176.61,31171.09,31178.16,31163.99,2020-01-01
2,TYP1,19673.13,19705.02,19729.79,19711.92,19736.99,19733.78,19732.99,19557.83,19252.19,...,18990.05,19090.13,19180.85,19149.85,19248.76,19558.21,19687.12,19683.84,19638.56,2020-01-02
3,TYP2,31129.98,31145.05,31187.65,31193.19,31228.66,31270.21,31265.45,31276.15,31131.89,...,30436.53,30716.73,30953.29,30978.81,31428.59,31211.40,31136.95,31131.77,31136.45,2020-01-02
4,TYP1,19592.70,19535.51,19500.42,19694.35,19721.16,19730.49,19737.64,19627.18,19047.52,...,18488.53,18890.49,18984.58,19023.81,19178.64,19226.40,19494.24,19602.41,19502.27,2020-01-03
5,TYP2,31131.06,31148.09,31175.90,31141.59,31127.29,31193.22,31191.98,31167.16,30883.65,...,29513.78,30257.32,30580.36,30825.99,31102.23,31125.12,31118.11,31118.72,31125.68,2020-01-03
6,TYP1,19686.52,19621.22,19630.42,19710.61,19688.69,19626.41,19536.95,19400.52,19075.79,...,18706.08,18810.89,18850.17,19190.59,19381.88,19556.37,19719.62,19767.16,19733.33,2020-01-04
7,TYP2,31140.48,31148.22,31147.55,31159.35,31141.43,31163.29,31167.64,31167.47,30946.41,...,29911.66,30234.48,30453.26,31148.27,31215.81,31212.43,31119.76,31139.44,31120.33,2020-01-04
8,TYP1,19727.82,19707.38,19714.79,19639.58,19705.45,19726.02,19734.29,19517.93,19151.84,...,19014.06,19104.36,19169.97,19112.98,19565.51,19710.92,19681.32,19714.77,19672.73,2020-01-05
9,TYP2,31125.51,31134.55,31148.74,31160.36,31147.30,31150.73,31171.72,31160.24,30916.85,...,30557.12,30742.99,30954.02,31141.25,31200.96,31121.52,31115.02,31212.49,31206.85,2020-01-05


In [23]:
df1 = consult.request_data("GeneIdea", 0, dt.date(2021, 1, 1), dt.date(2021, 1, 5),filtros=[''])
df1

,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,Values_Hour09,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,6660644.31,6471287.14,6236858.57,6052481.54,5934499.93,5828792.96,5439029.54,5473983.60,5763666.07,...,6517506.91,6474365.58,6526204.59,7420495.88,7659657.73,7523854.12,7310670.80,6965195.16,6666247.91,2021-01-01
1,Sistema,6347485.67,6076775.30,5928709.24,5848874.08,5842000.69,5862457.64,5791291.74,6144260.18,6604882.32,...,7448306.00,7359629.65,7392737.64,8251196.09,8438304.86,8245148.06,7924709.27,7461506.72,7028138.44,2021-01-02
2,Sistema,6664394.29,6360901.55,6216714.65,6125111.39,6096100.24,6091414.73,5886822.40,6068663.63,6415789.80,...,7267068.86,7133780.14,7201605.06,8138566.26,8386635.21,8264476.95,7984895.11,7565844.08,7092822.03,2021-01-03
3,Sistema,6695350.68,6451412.41,6288683.24,6210814.16,6301142.89,6482827.36,6622683.19,7166146.62,7745746.70,...,8873658.34,8676071.77,8453910.32,9174866.18,9341949.12,9131578.90,8708343.27,8056390.53,7457780.57,2021-01-04
4,Sistema,7006973.08,6724642.49,6556142.79,6462262.02,6524463.18,6699994.83,6845974.95,7375060.04,7901024.93,...,8996524.38,8796812.99,8605239.71,9322228.65,9444722.97,9196742.17,8770957.76,8124281.63,7497610.15,2021-01-05
